In [30]:
import pandas as pd
import numpy as np

In [2]:
data_train = pd.read_csv('train.tsv', sep='\t')
data_test = pd.read_csv('test.tsv', sep='\t')
data_val = pd.read_csv('val.tsv', sep='\t')

### Просмотрел внимательно все датасеты, провел анализ данных

Увидел зависимость между подстрокой файла и наличием вируса

In [4]:
data_train['is_virus'][data_train['filename'].str.contains(r'VirusShare')].value_counts()

is_virus
1    6848
Name: count, dtype: int64

In [5]:
data_train['is_virus'][data_train['filename'].str.contains(r'\.exe')].value_counts()

is_virus
0    5233
1    4012
Name: count, dtype: int64

In [6]:
data_train['is_virus'][data_train['filename'].str.contains(r'\+')].value_counts()

is_virus
1    2425
0       4
Name: count, dtype: int64

In [7]:
data_train['is_virus'][data_train['filename'].str.contains(r'\-')].value_counts()

is_virus
1    4012
0     692
Name: count, dtype: int64

### Отсортировал и сделал списки из библиотек в каждой строке столбца libs  

Из одной колонки libs получил 1924 колонок с названиями всех возможных бибилотек данной колонки. 0 в ячейке означает отсутвие библиотеки у файла, 1 означает ее наличие. 

In [5]:
from sklearn.preprocessing import MultiLabelBinarizer

# Сортирую библиотеки и разделяю их на список подстрок  
data_train['libs'] = data_train['libs'].apply(lambda x: sorted(x.split(',')))

# Преобразовываю колонку со всеми библиотеками в колонки с отдельными библотеками, применяя бинаризацию категориальных данных  
mlb = MultiLabelBinarizer()
enc_libs = pd.DataFrame(mlb.fit_transform(data_train['libs']), columns=mlb.classes_, index=data_train.index)

data_train = pd.concat([data_train, enc_libs], axis=1)

data_train = data_train.drop('libs', axis=1)

С помощью анализа данных в колонке filename проделываю похожую процедуру с данной колонкой, создавая 4 дополнительных колонки. 0 в ячейке означает отсутвие подстроки у названия файла, 1 означает ее наличие.

In [7]:
str_filenames = ['VirusShare', '.exe', '+', '-']


for j in str_filenames:
	data_train[j] = [1 if j in data_train['filename'][i] else 0 for i in range(len(data_train))]
data_train = data_train.drop(columns='filename')

### Теперь провжу точно такую же процедуру с валидационным датафреймом, чтобы не возникало ошибок при предсказании данных для валидационного датасета

In [8]:
data_val['libs'] = data_val['libs'].apply(lambda x: sorted(x.split(',')))

enc_libs = pd.DataFrame(mlb.fit_transform(data_val['libs']), columns=mlb.classes_, index=data_val.index)

data_val = pd.concat([data_val, enc_libs], axis=1)

data_val = data_val.drop('libs', axis=1)

In [9]:
for j in str_filenames:
	data_val[j] = [1 if j in data_val['filename'][i] else 0 for i in range(len(data_val))]
data_val = data_val.drop(columns='filename')

### Нахожу пересечение (наличие их в одном и другом датафрейме) названий колонок в тренировочном и валидационном датафреймах и удаляю все лишние колонки, чтобы избежать ошибок при предсказании данных  

In [10]:
common_columns = data_train.columns.intersection(data_val.columns)
data_train = data_train[common_columns]

In [11]:
X_train = data_train.drop(columns='is_virus')
y_train = data_train['is_virus']

In [12]:
common_columns = data_val.columns.intersection(data_train.columns)
data_val = data_val[common_columns]

In [13]:
X_val = data_val.drop(columns='is_virus')
y_val = data_val['is_virus']

### Для обучения модели интуитивно решил выбрать метод опорных векторов 

In [14]:
from sklearn.svm import SVC

In [15]:
svm = SVC() 
svm.fit(X_train, y_train)

SVC()

In [16]:
pred = svm.predict(X_val)

Ради эксперимента решил поработать и подобрать параметры для SVM, но по итогу модель показала наилучший результат без вводимых параметров. 

In [20]:
# from sklearn.model_selection import GridSearchCV

# tuned_parameters = [{'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'gamma': [1e-3, 1e-4], 
# 											'C': [1, 10, 100, 1000], 'class_weight': [None, 'balanced'],]

# cv = GridSearchCV(SVC(), tuned_parameters, refit=True, verbose=3)

In [21]:
# cv.fit(X_train, y_train)

In [22]:
# cv.best_params_

In [17]:
from sklearn.metrics import f1_score, confusion_matrix, classification_report

f1_score(y_val, pred)

0.9894606323620583

In [18]:
print(classification_report(y_val, pred, digits=4))

              precision    recall  f1-score   support

           0     0.9948    0.9625    0.9784       400
           1     0.9815    0.9975    0.9895       800

    accuracy                         0.9858      1200
   macro avg     0.9882    0.9800    0.9839      1200
weighted avg     0.9860    0.9858    0.9858      1200



In [19]:
tn, fp, fn, tp = confusion_matrix(pred, y_val).ravel()
print(f'true positive: {tp}')
print(f'false positive: {fp}')
print(f'true negative: {tn}')
print(f'false negative: {fn}')

true positive: 798
false positive: 2
true negative: 385
false negative: 15


### Далее подготавливаю тестовые данные по ранее описанным операциям для предсказания результатов 

In [20]:
data_test['libs'] = data_test['libs'].apply(lambda x: sorted(x.split(',')))

enc_libs = pd.DataFrame(mlb.fit_transform(data_test['libs']), columns=mlb.classes_, index=data_test.index)

data_test = pd.concat([data_test, enc_libs], axis=1)

data_test = data_test.drop('libs', axis=1)

In [21]:
# X_train = data_train.drop(columns=['is_virus', 'VirusShare', '.exe', '+',	'-'])
y_train = data_train['is_virus']

In [22]:
common_columns = data_train.columns.intersection(data_test.columns)
data_train = data_train[common_columns]

In [23]:
common_columns = data_test.columns.intersection(data_train.columns)
data_test = data_test[common_columns]

In [25]:
# svm = SVC() 
svm.fit(data_train, y_train)

SVC()

Понимаю, что на тестовых данных точность модели и результат будут ниже, чем на валидационных, поскольку в тестовом датасете отсутсвует колонка filename, соответственно, зависимостей при обучении становится меньше 

In [26]:
pred = svm.predict(data_test)

Создаю файл txt с предсказанными данными 

In [27]:
# with open('prediction.txt', 'w') as f:
#     f.write(f'{list(pred)}')

In [31]:
np.savetxt('prediction.txt', pred, fmt='%d')